# Hyperparameter tuning
In this laboratory, you will train an MLP model on a dataset of benign and DDoS network traffic using an automated mechanism to find the best hyper-parameters. This laboratory consists of a series of implementation steps:
- add an early-stopping strategy to the grid-search mechanism
- add more folds
- implement a randomised search with sci-kit libraries using a continuous variable for the learning rate
- compare Grid Search with Randomized Search in terms of execution time and accuracy of the best model on the validation set
- replace the model architecture with a CNN and the relevant hyperparameters to tune (e.g., number of kernels, kernel height, max pooling, etc)

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Lecture on Intrusion Detection with Deep Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import sys
import time
import glob
import argparse
import tensorflow as tf
import numpy as np
import random as rn
import os
import csv
import h5py
import logging
import warnings

# Seed Random Numbers
SEED = 1
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)
config = tf.compat.v1.ConfigProto(inter_op_parallelism_threads=1)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=DeprecationWarning)

from itertools import cycle
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Activation,  Flatten, Conv2D
from tensorflow.keras.layers import  GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, save_model, load_model, clone_model
from sklearn.metrics import f1_score, precision_score, accuracy_score, log_loss, confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

import tensorflow.keras.backend as K
tf.random.set_seed(SEED)
K.set_image_data_format('channels_last')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.log_device_placement = True  # to log device placement (on which device the operation ran)

In [ ]:
# hyperparameters
MAX_EPOCHS=20
hyperparamters = {
    "dense_layers" : [1,2],
    "n_neurons" : [16,32]
}

In [ ]:
def load_dataset(path):
    filename = glob.glob(path)[0]
    dataset = h5py.File(filename, "r")
    set_x_orig = np.array(dataset["set_x"][:])  # features
    set_y_orig = np.array(dataset["set_y"][:])  # labels

    X = np.reshape(set_x_orig, (set_x_orig.shape[0], set_x_orig.shape[1], set_x_orig.shape[2], 1))
    Y = set_y_orig

    return X, Y

In [ ]:
# MLP model
def MLPModel(dense_layers=2, n_neurons=32,learning_rate=0.01,input_shape=(10,11,1)):
    K.clear_session()

    model = Sequential(name  = "mlp")
    model.add(Flatten(input_shape=input_shape))
    for layer in range(dense_layers):
        model.add(Dense(n_neurons, activation='relu', name='hidden-fc' + str(layer)))
    model.add(Dense(1, activation='sigmoid', name='fc2'))
    print(model.summary())
    compileModel(model, learning_rate)
    return model

In [ ]:
def compileModel(model,lr):
    optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])  # here we specify the loss function

In [ ]:
X_train, Y_train = load_dataset("../Datasets/IDS2017/*" + '-train.hdf5')
X_val, Y_val = load_dataset("../Datasets/IDS2017/*" + '-val.hdf5')

X_train, Y_train = shuffle(X_train, Y_train, random_state=SEED)
X_val, Y_val = shuffle(X_val, Y_val, random_state=SEED)

In [ ]:
# instantiate the model
keras_classifier = KerasClassifier(build_fn = MLPModel)

In [ ]:
# define the hyperparameter search strategy
rnd_search_cv = GridSearchCV(keras_classifier, hyperparamters, cv=2, return_train_score=True)

In [ ]:
# Start the training process
rnd_search_cv.fit(X_train, Y_train, epochs=MAX_EPOCHS, validation_data=(X_val, Y_val))

In [ ]:
# print the results
print("Train scores: ", rnd_search_cv.cv_results_)
best_model = rnd_search_cv.best_estimator_.model
Y_pred_val = (best_model.predict(X_val) > 0.5)
Y_true_val = Y_val.reshape((Y_val.shape[0], 1))
f1_score_val = f1_score(Y_true_val, Y_pred_val)

print("\nBest parameters: ", rnd_search_cv.best_params_)
print("F1 Score of the best model on the validation set: ", f1_score_val)